# Import libraries

In [1]:
import numpy as np
import cv2
import pkg_resources
import imutils
import numpy
from imutils.face_utils import rect_to_bb
import dlib
import time

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Sequential

from keras.models import load_model

# Load data

In [4]:
# Select wich model to run
models = [
    './Models/model_mobile_init/', #0
    './Models/model_mobile_best/', #1
    './Models/model_VGG19/', #2
    './Models/model_mobile_3_class/' # 3
    
]

model_num = 1 #change model here
img_height = 128
img_width = 128

# Create a detector with Dlib
detector = dlib.get_frontal_face_detector()

# Load the model with this model
model = load_model(models[model_num]+'Model/')
model.load_weights(models[model_num]+'checkpoint/checkpoint')

2022-06-21 01:52:42.111173: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ./Models/model_mobile_best/checkpoint/checkpoint: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


OSError: Unable to open file (file signature not found)

# Detector de Barbijos

In [6]:
# Open the camera
cam = cv2.VideoCapture(0)
while cam.isOpened():
    ret, img = cam.read()
    
    img_p = img
    arr = ['Mask','No mask']
    arr_C = [(0,255,0),(0, 0, 255)]
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 1
    color = (255, 255,255)
    thickness = 1
    
    # Detect how many people in the frame Dlib
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dets = detector(gray)

    # If more than 3 people in the frame skip the model so the system doesn't freeze
    if(len(dets) < 3):
        for face in dets:
            x= face.left()
            y= face.top()
            w= face.width()
            h= face.height()
            org1 = (x,y+h+30)
            org2 = (x,y+h+60)
        
            crop = img_p[y:y+h, x:x+w]
            
            # Prepare the image for the model
            img_p1 = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
            img_p1 = cv2.resize(crop,(img_width, img_height))
            img_p1 = img_p1[None, :]
        
            # Time for prediction
            start = time.time()

            # Make a prediction
            prediction = model.predict(img_p1,verbose=0)
        
            end = time.time()
            
            print('time to predict:',end - start)

            pos = numpy.argmax(prediction)
            
            # Compensate for bias
            if pos == 0:
                if prediction[0][pos] < 0.97:
                    pos = 1
        
            por = "{:.2f}".format(prediction[0][pos]*100)
            text1 = arr[pos]
            text2 = str(por) +'%'
    
            # Show face rectangle
            cv2.rectangle(img, (x, y), (x+w, y+h), arr_C[pos], 2)
        
            # Show result
            cv2.putText(img, text1, org1, font, 
                        fontScale, color, thickness, cv2.LINE_AA)
            
            # Show certainty 
            cv2.putText(img, text2, org2, font, 
                        fontScale, color, thickness, cv2.LINE_AA)
    
    else:
        cv2.putText(img, 'More than 3 people', (50,50), font, 
                        fontScale, color, thickness, cv2.LINE_AA)
        
    img_mos = cv2.resize(img, (1200,800))
    cv2.imshow('live_feed',img_mos)
    k = cv2.waitKey(30) & 0xff
    if k == 27: # press 'ESC' to exit
        cam.release()

# Close Window
cv2.destroyAllWindows()

2022-06-21 01:52:49.316877: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


time to predict: 2.989804267883301
time to predict: 0.0658714771270752
time to predict: 0.05767011642456055
time to predict: 0.060207366943359375
time to predict: 1.474210500717163
time to predict: 0.05566859245300293
time to predict: 0.054296016693115234
time to predict: 0.05833125114440918
time to predict: 0.057711124420166016
time to predict: 0.05195045471191406
time to predict: 0.05991077423095703
time to predict: 0.08898019790649414
time to predict: 0.06406044960021973
time to predict: 0.08060503005981445
time to predict: 0.06653070449829102
time to predict: 0.0763394832611084
time to predict: 0.05779123306274414
time to predict: 0.05272650718688965
time to predict: 0.06253719329833984
time to predict: 0.05850410461425781
time to predict: 0.05948519706726074
time to predict: 0.05731797218322754
time to predict: 0.06414437294006348
time to predict: 0.06535458564758301
time to predict: 0.053495168685913086
time to predict: 0.057749032974243164
time to predict: 0.06201004981994629
ti